In [2]:
import numpy as np
import geopy

The goal is to estimate the distance traveled so far in a given lap, assuming no lane changes, and that the human driver would keep the vehicle on lane-center with minimal deviation from lap to laps. 

Total track distance:
- left lane: 3709.23 m
- right lane: 3825.92 m

Approach 1: Speed integration

Assume that we have accurate vehicle longitudinal speed from CAN reading
- sampling frequency: 10hz